In [ ]:
import sys
import os
import re
import copy
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import math 
import random
import itertools
from typing import List, Optional 
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate, optimize
from tqdm.notebook import tqdm

# load the various FastSim libraries
import DetectorSimulation.Detector as Detector
import DetectorSimulation.Particle as Particle
import DetectorSimulation.Vertex as Vertex
import DetectorSimulation.lorentz_transformation as lt
import DetectorSimulation.llp_gun as llp_gun
## depending on which fvs are used load a different specialized llp gun (llp_gun_new) (built on top of llp_gun)
import DetectorSimulation.llp_gun_new as lg
## NOTE: the various hadronic functions in llp_gun_new must be updated with the correct path of the LLP hadronic decay 
## 4-vector files depending on the type of analysis 

## These hadronic functions are almost the same but were separated into different functions for ease of use with the 
## different LLP analyses - but should be very easy to update them for any new analysis that need be done

# load helper function library for later
from Helpers.functions import *


### IMPORTANT ###
# All the hadronic decay 4-vector files are in the rest frame of the parent LLP (see files) and the code is set-up to boost them 
# to lab frame and deal with the same

# To use hadronic decay 4-vector files not in the rest frame of the parent LLP, one needs to comment/uncomment certain lines 
# in the llp_gun.py file 

## We give a demo of some basic functionalities. We also give a first look at the reconstruction/trigger capabilities of FastSim

## LLP Particle manually

In [ ]:
## LLP PARTICLE MANUALLY

# Let's start with some code taken from the get_weight function itself

## First, let us setup the detector by choosing a param_card
param_card = "param_card_CDR.txt"
# change this to the path to param_card on your local computer
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_FastSim/" + param_card

# setup detector
detector_benchmark = Detector.Detector(path_to_param_card)
# lets see what the detector configuration is (might be messy, feel free to comment out)
print(detector_benchmark.config)

# clear the detector of past events, if any
detector_benchmark.clear_detector()

# choose a position for the LLP to start at - we just choose the interaction point (IP) and the sim
# automatically uses kinematics to figure out trajectory
position = (0,0,0)

# create an LLP particle, Particle.Particle(position, 4-vector, PID)
# play around with various 4-vectors and positions!
four_p = (100, 0, 100, 60) # (E, px, py, pz)
pid = 13 # let's say it is a muon (try changing the pid to an inivisble particle, say PID = 12, electron-neutrino)

llp = Particle.Particle(position, four_p, pid)

# feed this particle to the detector as a new (particle) event - other options include new_vertex_event which 
# feeds a vertex (i.e. an LLP with its decay products) to the detector
detector_benchmark.new_particle_event(llp)

# show/save event display (if needed) using the following (MORE information in the next few blocks)
# setting argument show = True displays and saves
# show = False saves but does not display
filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# now lets get some information about this particle after it passed through detector

# get the current particle in the detector
par = detector_benchmark.return_current_particle() 
# just print the object to get info
print(par)

## LLP Vertex manually

In [ ]:
# LLP VERTEX MANUALLY

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_FastSim/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)
detector_benchmark.clear_detector()

position = (0,0,0)
four_p = (100, 0, 100, 60) # random 4-vector (might not equal sum of daughter 4-vectors, but we use it for demo anyway)
pid = 1023

# create daughter particles for the vertex 
daughter1 = Particle.Particle(position, (6, 0.2, -2.1, 3.7), 13)
daughter2 = Particle.Particle(position, (0.7, 0, 0.3, 0.4), 211)
daughter3 = Particle.Particle(position, (2, 0.6, 1, -0.2), 13)

decay_products = [daughter1, daughter2, daughter3]

# create the vertex
vertex = Vertex.Vertex(position, four_p, pid, decay_products)
detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# now lets get some information about this vertex after it passed through detector

# get the current vertex in the detector
ver = detector_benchmark.return_current_vertex() 
# just print the object to get info
print(ver)

## LLP Vertex from file

In [ ]:
# LLP VERTEX FROM FILE

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_FastSim/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)

##################
# change this to the path to bb_15.txt on your local machine
path_to_file = "/Users/jai/Desktop/MATHUSLA_FastSim/bb_15.txt"
position = (0,0,0)
num = 2 # number of vertices you want to be read in from the given file

# get vertex (in LLP rest frame)
vertices = llp_gun.create_llp_from_file(path_to_file, position, num) # returns a list

# let's choose one of these, say the first one
vertex = vertices[0]

# feed it in 
detector_benchmark.clear_detector()
detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
print("Non-boosted LLP vertex\n")
detector_benchmark.detector_display(filename, show=True)
# in some cases you might notice "Particle trajectory out of bound." messages printed, 
# because the daughter particles from the LLP go in the direction opposite/out of the display frame

# what if we want to boost the vertex to say some detector frame
# we can aim the LLP at some specific starting target and boost LLP momentum to new momentum if specified as follows
target = (0,120,70)
p_norm = 100 # momentum LLP vertex (i.e. decay products) should be boosted to, we use the masses buit into 
#              vertex.decay_product to get the boost for each daughter
boosted_vertex = llp_gun.align_trajectory(vertex, target, p_norm)

# feed in vertex
detector_benchmark.clear_detector()
detector_benchmark.new_vertex_event(boosted_vertex)
# as expected we should end up with all the daughters (there should be more than the 3 visible before!) 
# boosted forward as expected

filename = "/Users/jai/Desktop/hello_MATHUSLA"
print("Boosted LLP vertex\n")
detector_benchmark.detector_display(filename, show=True)

## if we look at the top right figure, it is clear that there are tracks passing through ALL the ceiling sensor planes,
## however the top middle figure does not show the same picture, in fact, it might seem as if the tracks do not pass
## through ALL the ceiling sensors -- THIS IS JUST AN ARTIFACT OF THE MATPLOTLIB LIBRARY NOT BEING ABLE TO PLOT 2D
## PROJECTIONS OF 3D OBJECTS "NICELY", AND AS SUCH OVERLAYS/INTERSECTIONS ARE NOT EXACT ENOUGH - in this figure,
## the detector planes always overlay the tracks

## there is a work-around in the code and can be accessed by setting the argument zorder in the detector_display
## function to zorder = 1000 (so zorder only takes TWO values, 10 (default) or 1000)

detector_benchmark.detector_display(filename, show=True, zorder=1000)

## as you might see, now it's the tracks that overlay the detector planes

## both these views are provided as a sanity check for the user.


## Reconstruction/Trigger Criteria

In [ ]:
## CHECKING IF RECONSTRUCTION/TRIGGER CRITERIA ARE PASSED
## by now we know how to create an LLP vertex, let's learn how to check if it is actually "detected" or not

param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_FastSim/" + param_card

detector_benchmark = Detector.Detector(path_to_param_card)
detector_benchmark.clear_detector()

position = (0,0,0)
pid = 1023
four_p = (100, 0, 100, 60)

daughter1 = Particle.Particle(position, (6, 0.2, -2.1, 3.7), 13)
daughter2 = Particle.Particle(position, (0.7, 0, 0.3, 0.4), 211)
daughter3 = Particle.Particle(position, (2, 0.6, 1, -0.2), 13)

decay_products = [daughter1, daughter2, daughter3]


vertex = Vertex.Vertex(position, four_p, pid, decay_products)
# let's say the LLP vertex decayed at the point (0,120,70) with p_norm = 100
target = (0,120,70)
p_norm = 100 
vertex = llp_gun.align_trajectory(vertex, target, p_norm)

detector_benchmark.new_vertex_event(vertex)

filename = "/Users/jai/Desktop/hello_MATHUSLA"
detector_benchmark.detector_display(filename, show=True)

# upto this point everything was same as before

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2"): # gives 0 or 1 
    print("passed DV2")
else:
    print("did not pass DV2")

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium3"): # gives 0 or 1 
    print("passed DV3")
else:
    print("did not pass DV3")

# how to check if LLP vertex passed trigger (nearest_neighbour, see paper) criteria
# NOTE: we use detector_benchmark.event_pass_trigger(trigger_criteria) without any argument as there is only one criteria
# and the simulation automatically uses that as default
if detector_benchmark.event_pass_trigger():
    print("passed trigger")
else:
    print("did not pass trigger")
    
# Finally we can take a product of recon and trigger to find whether LLP vertex passed both
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2") * detector_benchmark.event_pass_trigger():
    print("passed both recon + trigger")
else:
    print("did not pass recon or trigger or maybe both")

## Now we give an explicit demo - in essence a mini-version of the pipeline used to generate our results

NOTE: Once again, a lot of the parts of the block below can be used as independent functions by themselves

In [ ]:
# choose a mass for the LLP (in GeV)
m = 1.0

# say we read in ctau from a file
ctau = 100

# choose a param card - we choose the default CDR geometry as before
param_card = "param_card_CDR.txt"
path_to_param_card = "/Users/jai/Desktop/MATHUSLA_FastSim/" + param_card

# let's say we are considering SM+S produced LLP decay to hadrons. We read in the possible decay products from the
# file "hadrons1to2gev_1.0.txt" and use them to construct vertices and shoot them at MATHUSLA

# first, read in say 20 (or however many you might want) LLP 4-vectors from "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"
fv_path = "/Users/jai/Desktop/MATHUSLA_FastSim/SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"

# use the read_vectors_SMS file to read in, as format of 4-vectors is (weight, E, px, py, pz) - we ignore the weight in this the demo
vectors = read_vectors_SMS(fv_path, 20) # 4-vectors from B parents

# note length of read-in 4-vector list as we use this to iterate through this list later
num_iter = len(vectors)

# finally run the simulation - setup the detector by giving it the path to the chosen param_card
# change path to local machine
detector_benchmark = Detector.Detector(path_to_param_card)

# find max min angles OF THE DECAY VOLUME (note its not the detector volume!) 
x = np.array([detector_benchmark.config.decay_x_min, detector_benchmark.config.decay_x_max])
y = np.array([detector_benchmark.config.decay_y_min, detector_benchmark.config.decay_y_max])
z = np.array([detector_benchmark.config.decay_z_min, detector_benchmark.config.decay_z_max])

corners = np.array(np.meshgrid(x, y, z)).T.reshape(-1,3)
points = corners.copy()

detector_theta = []
detector_phi = []

# for efficiency, randomly generate 100 points lying in/on the detector and take min,max angles from these
# -- ends up giving the (almost) actual min, max (using Central Limit Theorem)
for j in itertools.product(corners, corners):
    for k in range(100):
        u = random.uniform(0,1)
        new = u * j[0] + (1-u) * j[1]
        detector_theta.append(get_theta(new))
        detector_phi.append(get_phi(new))

theta_min, theta_max = min(detector_theta), max(detector_theta)
phi_min, phi_max = min(detector_phi), max(detector_phi)

# now we have angle ranges for MATHUSLA and can compare LLP 4-vectors (with starting point being the IP)
# with the same to check if they pass through the detector decay volume


################## actual simulation ################ 

# iterate through the LLP 4-vectors
for k in tqdm(range(num_iter)): 
    print("LLP",k,"\n")
    # forget about first entry i.e. weight and onyl keep track of actual 4-vector
    four_p = vectors[k][1:]
    # get the LLP theta and phi angles 
    llp_theta = get_theta(four_p[1:])
    llp_phi = get_phi(four_p[1:])
    # check if LLP passes theta (equivalently eta) range of MATHUSLA i.e. it has chance of passing through
    ## (NOTE: while theta check should ensure that LLP SHOULD pass through, sometimes it still might not
    #       due on numerical precision etc - all these sanity checks are built in at every stage)
    if (llp_theta > theta_min) and (llp_theta < theta_max):
        # if it does, randomly rotate the phi of the LLP 4-vector
        four_p_rot = deal_with_phi(four_p, phi_min, phi_max)
        # use new 4-vector, lifetime of LLP (say 100m), and the detector to generate
        # weight, decay position, boost of the LLP
        pack = get_weight(four_p_rot, m, ctau, detector_benchmark) # returns pack = (weight, decay_position, boost)           

        # if get_weight does not return None i.e. LLP enter and exits detector (and there is no numerical precision issue etc.)
        if pack is not None:
            print("LLP passed through MATHUSLA")
            # once again, clear the detector of previous events (if any)
            detector_benchmark.clear_detector()

            # read-in LLP vertex from file 
            llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"hadrons1to2gev")

            # make sure LLP is not None i.e. not an invisible decay
            if llp is not None:
                detector_benchmark.new_vertex_event(llp)
                
                filename = "/Users/jai/Desktop/hello_MATHUSLA"
                detector_benchmark.detector_display(filename, show=True)
                
                # the sim has run now, so we can check if LLP was reconstructed using different criteria
                
                print("LLP DV2 recon:", detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2"))
                print("LLP DV3 recon:", detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium3"))
                print("LLP trigger:", detector_benchmark.event_pass_trigger(),"\n")
                
                
            else:
                print("Invisible decay\n")
                
        else:
            print("LLP did not pass through MATHUSLA\n")

## Bonus: Converting hadronic decay 4-vectors into Fastim format

While we have provided links to repositories containing the hadronic decay products required for various LLP analyses, the FastSim takes them as input in a specific format (for e.g. look at file "bb_15.txt" or "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"). Here we demo the script to convert the given files into such a format.

In [ ]:
from Helpers.script import *

# we take in a hadronic decay 4-vectors file similar to the ones at the github links in the paper and 
# convert it into the format needed for input into the FastSim

# first argument is where to save output, second argument is where to read in the file to be converted
# change paths as needed
write_hadrons_for_fastsim("/Users/jai/Desktop/MATHUSLA_FastSim/sample_fastsim_input.txt", "/Users/jai/Desktop/MATHUSLA_FastSim/to_be_converted_to_fastsim_input_format.txt")